# Using Airbnb data to analyze the market in Seattle
I did the analysis from the point of view of a Seattle homeowner. The main objective of a Seattle homeowner would be to maximize revenues by attracting customers willing to pay high prices. This results in the following qustions:

### 1. Business Questions

#### 1.1 What drives higher ratings ?

#### 1.2 Where are the highest prices for rentals ?

#### 1.3 When are the highest prices for rentals ?

### 2. Data Understanding

#### 2.1 Retrieve Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython import display
import collections
from itertools import chain
import sklearn
from time import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
%matplotlib inline

In [2]:
df_listings = pd.read_csv("data/listings.csv")
df_calendar = pd.read_csv("data/calendar.csv")
df_reviews = pd.read_csv("data/reviews.csv")

#### 2.2 Overview and Exploration

In [3]:
# Set some display options
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 2000)

In [20]:
# Check number of rows and columns
print(df_listings.shape)
print(df_calendar.shape)
print(df_reviews.shape)

(3818, 86)
(1393570, 4)
(84849, 6)


In [21]:
# Get column names
print(df_listings.columns.values)
print(df_calendar.columns.values)
print(df_reviews.columns.values)

['id' 'listing_url' 'scrape_id' 'last_scraped' 'name' 'summary' 'space'
 'description' 'experiences_offered' 'transit' 'thumbnail_url'
 'medium_url' 'picture_url' 'xl_picture_url' 'host_id' 'host_url'
 'host_name' 'host_since' 'host_location' 'host_about'
 'host_response_time' 'host_response_rate' 'host_acceptance_rate'
 'host_is_superhost' 'host_thumbnail_url' 'host_picture_url'
 'host_neighbourhood' 'host_listings_count' 'host_total_listings_count'
 'host_verifications' 'host_has_profile_pic' 'host_identity_verified'
 'street' 'neighbourhood' 'neighbourhood_cleansed'
 'neighbourhood_group_cleansed' 'city' 'state' 'zipcode' 'market'
 'smart_location' 'country_code' 'country' 'latitude' 'longitude'
 'is_location_exact' 'property_type' 'room_type' 'accommodates'
 'bathrooms' 'bedrooms' 'beds' 'bed_type' 'amenities' 'price'
 'security_deposit' 'cleaning_fee' 'guests_included' 'extra_people'
 'minimum_nights' 'maximum_nights' 'calendar_updated' 'has_availability'
 'availability_30' 'avail

In [22]:
df_listings.describe()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,guests_included,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month
count,3.818000e+03,3.818000e+03,3.818000e+03,3816.000000,3816.000000,3818.000000,3818.000000,3818.000000,3802.000000,3812.000000,3817.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3171.000000,3160.000000,3165.000000,3160.000000,3167.000000,3163.000000,3162.000000,3818.000000,3191.000000
mean,5.550111e+06,2.016010e+13,1.578556e+07,7.157757,7.157757,47.628961,-122.333103,3.349398,1.259469,1.307712,1.735394,1.672603,2.369303,780.447617,16.786276,36.814825,58.082504,244.772656,22.223415,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,9.452245,2.946307,2.078919
std,2.962660e+06,0.000000e+00,1.458382e+07,28.628149,28.628149,0.043052,0.031745,1.977599,0.590369,0.883395,1.139480,1.311040,16.305902,1683.589007,12.173637,23.337541,34.063845,126.772526,37.730892,6.606083,0.698031,0.797274,0.595499,0.568211,0.629053,0.750259,5.893029,1.822348
min,3.335000e+03,2.016010e+13,4.193000e+03,1.000000,1.000000,47.505088,-122.417219,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,2.000000,3.000000,2.000000,2.000000,4.000000,2.000000,1.000000,0.020000
25%,3.258256e+06,2.016010e+13,3.275204e+06,1.000000,1.000000,47.609418,-122.354320,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,60.000000,2.000000,13.000000,28.000000,124.000000,2.000000,93.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,1.000000,0.695000
50%,6.118244e+06,2.016010e+13,1.055814e+07,1.000000,1.000000,47.623601,-122.328874,3.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1125.000000,20.000000,46.000000,73.000000,308.000000,9.000000,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000,1.540000
75%,8.035127e+06,2.016010e+13,2.590309e+07,3.000000,3.000000,47.662694,-122.310800,4.000000,1.000000,2.000000,2.000000,2.000000,2.000000,1125.000000,30.000000,59.000000,89.000000,360.000000,26.000000,99.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,2.000000,3.000000
max,1.034016e+07,2.016010e+13,5.320861e+07,502.000000,502.000000,47.733358,-122.240607,16.000000,8.000000,7.000000,15.000000,15.000000,1000.000000,100000.000000,30.000000,60.000000,90.000000,365.000000,474.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,37.000000,12.150000


There are 3 datasets:

- listings
- calendar
- reviews

The most relevant dataset for our analysis is the listings dataset. The calendar dataset is relvant for answering the third question about popular times.

Since the reviews dataset is mainly unstructured data, we will postpone any analysis.

#### 2.2 Treatment of missing data

In [23]:
# Check for missing values in listings
(df_listings.isnull().sum()/len(df_listings)).sort_values(ascending=False)

transit                             0.244631
host_about                          0.224987
host_acceptance_rate                0.202462
review_scores_accuracy              0.172342
review_scores_checkin               0.172342
review_scores_value                 0.171818
review_scores_location              0.171556
review_scores_cleanliness           0.171032
review_scores_communication         0.170508
review_scores_rating                0.169460
first_review                        0.164222
last_review                         0.164222
reviews_per_month                   0.164222
space                               0.149031
host_response_rate                  0.136983
host_response_time                  0.136983
neighbourhood                       0.108958
thumbnail_url                       0.083814
medium_url                          0.083814
xl_picture_url                      0.083814
host_neighbourhood                  0.078575
summary                             0.046359
bathrooms 

In [24]:
# Check columns >=25% missing values
[cols for cols in df_listings.columns.values if (df_listings[cols].isnull().sum()/len(df_listings))>=0.25]

[]

In [25]:
# Distribution of missing values.
(df_listings.isnull().sum()/len(df_listings)).describe()

count    86.000000
mean      0.038222
std       0.069719
min       0.000000
25%       0.000000
50%       0.000000
75%       0.035817
max       0.244631
dtype: float64

There are a number of columns containing missing values. The license column has no values at all.

There are no missing values in the price column, perhaps this column can be used instead of weekly and monthly prices. But there seems to be no good substitute for square_feet.

In [26]:
df_listings[['monthly_price', 'weekly_price', 'price']].head(10)

KeyError: "['monthly_price' 'weekly_price'] not in index"

In [71]:
df_listings[['monthly_price', 'weekly_price', 'price']].replace('[\$,]','',regex=True).astype(float).corr()

,monthly_price,weekly_price,price
monthly_price,1.000000,0.942644,0.873450
weekly_price,0.942644,1.000000,0.937861
price,0.873450,0.937861,1.000000


The price field seems to indicate a daily price, therefore I will drop the monthly and weekly prices from the dataset due to a large number of missing values. The price field should act as a good substitute.

In [10]:
df_listings[df_listings['square_feet'].notnull()][['square_feet', 'bathrooms', 'bedrooms', 'beds']].corr()

,square_feet,bathrooms,bedrooms,beds
square_feet,1.000000,0.381094,0.448786,0.312155
bathrooms,0.381094,1.000000,0.418992,0.303472
bedrooms,0.448786,0.418992,1.000000,0.742920
beds,0.312155,0.303472,0.742920,1.000000


The bathrooms and bedroom field will be used as substitutes for the mostly missing square_feet field.

In [11]:
df_listings[['security_deposit']].head(10)

,security_deposit
0,NaN
1,$100.00
2,"$1,000.00"
3,NaN
4,$700.00
5,NaN
6,$150.00
7,$150.00
8,NaN
9,$100.00


A missing security deposit can probably replaced by a value of 0.

In [12]:
df_listings[['neighborhood_overview']].head(10)

,neighborhood_overview
0,NaN
1,"Queen Anne is a wonderful, truly functional village. You can walk everywhere... for coffee, for groceries, for a night out, for breathtaking views of the Puget Sound."
2,"Upper Queen Anne is a charming neighborhood full of a mix of beautifully maintained, turn-of-the-century craftsman homes, tudors and modern homes. The ""main drag"" is Queen Anne Ave. with a great selection of boutiques, restaurants, coffee shops and bars. Upper Queen Anne is just 1.5 miles from downtown Seattle, and situated on a hilltop with stunning panoramic views in every direction -- downtown Seattle, the Space Needle and Mt. Rainier to the southeast, Elliott Bay to the southwest, the Olympic Mountains to the west and the Cascade Mountains to the east. If you're a walker, you'll love the 4-mile ""Crown of Queen Anne"" loop which takes in all of these views!"
3,NaN
4,"We are in the beautiful neighborhood of Queen Anne on 14th Ave. W. A 5 minute walk from Whole Foods Market. Take a 5 minute drive south or north and find yourself in the lively neighborhood of Belltown or charming neighborhood of Ballard (respectively). Upper Queen Anne shops and restaurants are a 20 minute walk or a quick 5 minute drive up the hill. The Seattle Center (The Space Needle, the Children's Museum, EMP, Pacific Science Center, Chihuly Garden and Glass) is a 7 minute drive."
5,This part of Queen Anne has wonderful views and quiet streets with parking. There are several beautiful places to walk nearby.
6,"Close restaurants, coffee shops and grocery stores all in easy walking distance from our house."
7,We are centrally located between Downtown and ballard with a Whole Foods right across the street.
8,"Walking Score: 92 4 blocks from Kerry Park Famous Seattle restaurants within 1 block The hill became a popular spot for the city's early economic and cultural elite to build their mansions, and the name derives from the architectural style typical of many of the early homes. According to Wikipedia, Queen Anne's demographic is power-singles, young unmarried professionals. It boasts a bustling main street with restaurants, bars, shopping, coffee shops, farmers market, Trader Joe's, and the famous Molly Moon's Ice Cream."
9,"Queen Anne is a wonderful, truly functional village. You can walk everywhere... for coffee, for groceries, for a night out, for breathtaking views of the Puget Sound."


In [13]:
df_listings[['notes']].head(10)

,notes
0,NaN
1,"What's up with the free pillows? Our home was renovated and restored with health & safety in mind, especially the elimination/reduction of indoor allergens, toxins & other pollutants. (For this reason, we have no carpets, just hardwood & slate floors, with radiant heat.) Pillows tend to house a LOT of allergens over time, even when just used by a single owner. The statistics make many people feel pretty queasy but, essentially, pillows become great breeding grounds for bacteria and dust mites, not to mention drool, hair oils and perspiration. Since most people sweat more than a pint a night, the pillow can also easily trap mildew and mold. We use dust mite protectors for mattresses and duvets, but haven't found any that work well for pillows that maintain comfort/elasticity without the use of outgassing materials. So, I let the guests take their pillows but ask that they leave the pillowcases. And if you leave the pillows behind, they'll be donated to a local women's shelter that"
2,"Our house is located just 5 short blocks to Top Pot Doughnuts, Tribunali pizza, Caffe Fiore, Trader Joes and Molly Moons. Just 8 blocks to Queen Anne Ave."
3,NaN
4,Belltown
5,Let me know if you need anything or have suggestions to improve guest stays.
6,The room now has a mini frig to keep your favorite beverage chilled to perfection!
7,There are three rentals in our back yard . If you are not ok with sharing .. this is not the rental for you. NOW WITH High speed WI-FI !
8,NaN
9,"What's up with the free pillows? Our home was renovated and restored with health & safety in mind, especially the elimination/reduction of indoor allergens, toxins & other pollutants. (For this reason, we have no carpets, just hardwood & slate floors, with radiant heat.) Pillows tend to house a LOT of allergens over time, even when just used by a single owner. The statistics make many people feel pretty queasy but, essentially, pillows become great breeding grounds for bacteria and dust mites, not to mention drool, hair oils and perspiration. Since most people sweat more than a pint a night, the pillow can also easily trap mildew and mold. We use dust mite protectors for mattresses and duvets, but haven't found any that work well for pillows that maintain comfort/elasticity without the use of outgassing materials. So, I let the guests take their pillows but ask that they leave the pillowcases. And if you leave the pillows behind, they'll be donated to a local women's shelter that"


Since the neighborhood_overviewa and notes columns are unstructured text, that would requiere a sentiment analysis, these columns will be dropped.

In [14]:
df_listings[['cleaning_fee']].head(10)

,cleaning_fee
0,NaN
1,$40.00
2,$300.00
3,NaN
4,$125.00
5,$40.00
6,NaN
7,$25.00
8,NaN
9,$40.00


A missing cleaning fee can probably replaced by a value of 0.

In [15]:
(df_calendar.isnull().sum()/len(df_calendar)).sort_values(ascending=False)

price         0.32939
available     0.00000
date          0.00000
listing_id    0.00000
dtype: float64

Again the price column is missing in a lot of rows (32%).

In [16]:
(df_reviews.isnull().sum()/len(df_reviews)).sort_values(ascending=False)

comments         0.000212
reviewer_name    0.000000
reviewer_id      0.000000
date             0.000000
id               0.000000
listing_id       0.000000
dtype: float64

There are no missing data for reviews except for a small number of missing comments.

In [17]:
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to everything!
1,7202016,39087409,2015-07-20,32440555,Frank,"Kelly has a great room in a very central location. \r\nBeautiful building , architecture and a style that we really like. \r\nWe felt guite at home here and wish we had spent more time.\r\nWent for a walk and found Seattle Center with a major food festival in progress. What a treat.\r\nVisited the Space Needle and the Chihuly Glass exhibit. Then Pikes Place Market. WOW. Thanks for a great stay."
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighborhood. This is the kind of apartment I wish I had!\r\n\r\nDidn't really get to meet Kelly until I was on my out, but she was always readily available by phone. \r\n\r\nI believe the only ""issue"" (if you want to call it that) was finding a place to park, but I sincerely doubt its easy to park anywhere in a residential area after 5 pm on a Friday"
3,7202016,40813543,2015-08-02,33671805,George,"Close to Seattle Center and all it has to offer - ballet, theater, museum, Space Needle, restaurants of all ilk just blocks away, and the Metropolitan (probably the coolest grocer you'll ever find). Easy to find and Kelly was warm, welcoming, and really interesting to talk to."
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating in a great neighborhood. She has some great coffee and while I wasn't around much during my stay the time I spent interacting with her was very pleasant. \r\n\r\nThe apartment is in a great location and very close to the Seattle Center. The neighborhood itself has a lot of good food as well!


For reasons described in section 2.1 the following columns will be dropped:
    
- neighborhood_overview
- notes
- square_feet
- weekly_price
- monthly_price,
- license

In [18]:
# Drop columns
df_listings.drop(columns=['neighborhood_overview', 'notes', 'square_feet', 'weekly_price', 'monthly_price', 'license'],inplace=True)

For reasons described in section 2.1 the missing values for cleaning_fee and security_deposit will be replaced with numeric 0.

In [19]:
# Fill missing values with 0
df_listings['security_deposit']=df_listings['security_deposit'].fillna(0)
df_listings['cleaning_fee']=df_listings['cleaning_fee'].fillna(0)

In [ ]:
# Since the id column has no meaningful content, this column will be dropped, too.
df_listings.drop(columns=['id'],inplace=True)